In [1]:
#one time install of torch, transformers
!pip install torch
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import datetime
import gc

In [3]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [4]:
VAERSDATA_2023_df = pd.read_csv(r'2023VAERSDATA.csv', encoding='latin1', low_memory=False)

In [5]:
VAERSSYMPTOMS_2023_df = pd.read_csv(r'2023VAERSSYMPTOMS.csv')

In [6]:
VAERSSYMPTOMS_2023_df['VAERS_ID'][0]

2547730

In [7]:
VAERSSYMPTOMS_2023_df

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,2547730,Blood pressure orthostatic abnormal,25.1,COVID-19,25.1,Coma,25.1,Computerised tomogram,25.1,Exposure to SARS-CoV-2,25.1
1,2547730,Head injury,25.1,Headache,25.1,Laboratory test,25.1,Magnetic resonance imaging,25.1,SARS-CoV-2 antibody test negative,25.1
2,2547730,SARS-CoV-2 test positive,25.1,Unresponsive to stimuli,25.1,X-ray,25.1,NaN,NaN,NaN,NaN
3,2547731,Product preparation issue,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2547732,Injury associated with device,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
26434,2583346,Blepharospasm,25.1,Blood potassium decreased,25.1,Computerised tomogram normal,25.1,Electrocardiogram normal,25.1,Hypoaesthesia,25.1
26435,2583346,Laboratory test abnormal,25.1,Magnetic resonance imaging normal,25.1,Muscle disorder,25.1,NaN,NaN,NaN,NaN
26436,2583348,Injection site pain,25.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26437,2583349,Injection site hypoaesthesia,25.1,Injection site pain,25.1,Muscular weakness,25.1,Pain,25.1,X-ray limb,25.1


In [8]:
# Uncomment if '2023VAERSSYMPTOMS_cleaned.csv' does not exist

#count = 0
#for index in VAERSSYMPTOMS_2023_df.index:
    #for i in range(1, 6):
        
        #out_list = []
        #out_list.append(VAERSSYMPTOMS_2023_df['VAERS_ID'][index])
        #out_list.append(VAERSSYMPTOMS_2023_df['SYMPTOM' + str(i)][index])
        
        #if count == 0:
            #pd.DataFrame([out_list], columns = ['VAERS_ID', 'SYMPTOM']).to_csv(r'2023VAERSSYMPTOMS_cleaned.csv', index = False)
            #count += 1
            #continue
        
        
        #pd.DataFrame([out_list], columns = ['VAERS_ID', 'SYMPTOM']).to_csv(r'2023VAERSSYMPTOMS_cleaned.csv', mode = 'a', index = False, header = False)
        #count += 1

In [9]:
VAERSSYMPTOMS_2023_cleaned = pd.read_csv(r'2023VAERSSYMPTOMS_cleaned.csv')

In [10]:
VAERSSYMPTOMS_2023_cleaned.shape

(132195, 2)

In [11]:
VAERSDATA_2023_with_SYMPTOMS_df  = VAERSDATA_2023_df.set_index('VAERS_ID').join(VAERSSYMPTOMS_2023_cleaned.set_index('VAERS_ID')).reset_index()

In [12]:
VAERSDATA_2023_with_SYMPTOMS_df = VAERSDATA_2023_with_SYMPTOMS_df[~VAERSDATA_2023_with_SYMPTOMS_df['SYMPTOM'].isna()]

In [13]:
VAERS_ID_sample = pd.Series(VAERSDATA_2023_with_SYMPTOMS_df['VAERS_ID'].unique())
VAERSDATA_2023_with_SYMPTOMS_df_sample = VAERSDATA_2023_with_SYMPTOMS_df[VAERSDATA_2023_with_SYMPTOMS_df['VAERS_ID'].isin(VAERS_ID_sample)]

In [14]:
VAERSDATA_2023_with_SYMPTOMS_df_sample.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,SYMPTOM
0,2547730,01/01/2023,DE,53.0,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,NaN,NaN,NaN,2,12/31/2022,NaN,NaN,NaN,none,Blood pressure orthostatic abnormal
1,2547730,01/01/2023,DE,53.0,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,NaN,NaN,NaN,2,12/31/2022,NaN,NaN,NaN,none,COVID-19
2,2547730,01/01/2023,DE,53.0,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,NaN,NaN,NaN,2,12/31/2022,NaN,NaN,NaN,none,Coma
3,2547730,01/01/2023,DE,53.0,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,NaN,NaN,NaN,2,12/31/2022,NaN,NaN,NaN,none,Computerised tomogram
4,2547730,01/01/2023,DE,53.0,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,NaN,NaN,NaN,2,12/31/2022,NaN,NaN,NaN,none,Exposure to SARS-CoV-2


In [15]:
VAERSDATA_2023_with_SYMPTOMS_df_sample.tail()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,SYMPTOM
132189,2583349,02/16/2023,CA,75.0,75.0,NaN,F,NaN,C/o L deltoid and shoulder pain with weakness ...,NaN,...,As above,NaN,NaN,2,02/16/2023,NaN,Y,NaN,Atorvastatin calcium; diltiazem,X-ray limb
132190,2583350,02/16/2023,NC,34.0,34.0,NaN,M,NaN,Liver and joint pain. My liver pain persists e...,NaN,...,NaN,NaN,NaN,2,02/16/2023,NaN,Y,NaN,NaN,Arthralgia
132191,2583350,02/16/2023,NC,34.0,34.0,NaN,M,NaN,Liver and joint pain. My liver pain persists e...,NaN,...,NaN,NaN,NaN,2,02/16/2023,NaN,Y,NaN,NaN,Hepatic mass
132192,2583350,02/16/2023,NC,34.0,34.0,NaN,M,NaN,Liver and joint pain. My liver pain persists e...,NaN,...,NaN,NaN,NaN,2,02/16/2023,NaN,Y,NaN,NaN,Hepatic pain
132193,2583350,02/16/2023,NC,34.0,34.0,NaN,M,NaN,Liver and joint pain. My liver pain persists e...,NaN,...,NaN,NaN,NaN,2,02/16/2023,NaN,Y,NaN,NaN,Ultrasound liver abnormal


In [16]:
VAERSDATA_2023_with_SYMPTOMS_df_sample['SYMPTOM_TEXT'] = VAERSDATA_2023_with_SYMPTOMS_df_sample['SYMPTOM_TEXT'].fillna('BLANK')

In [17]:
VAERSDATA_2023_with_SYMPTOMS_value_counts = VAERSDATA_2023_with_SYMPTOMS_df_sample['SYMPTOM'].value_counts()

In [18]:
VAERSDATA_2023_with_SYMPTOMS_value_counts[VAERSDATA_2023_with_SYMPTOMS_value_counts > 100]

Product storage error               4335
COVID-19                            3601
No adverse event                    3451
SARS-CoV-2 test positive            2237
Expired product administered        1717
                                    ... 
Sepsis                               106
Loss of consciousness                105
Swelling                             103
Sneezing                             103
Incomplete course of vaccination     102
Name: SYMPTOM, Length: 112, dtype: int64

In [19]:
VAERSDATA_2023_with_SYMPTOMS_value_counts_100_up = VAERSDATA_2023_with_SYMPTOMS_value_counts[VAERSDATA_2023_with_SYMPTOMS_value_counts > 100].index

In [20]:
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned = pd.DataFrame()

VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['ID_txt_zip'] = pd.Series(zip(VAERSDATA_2023_with_SYMPTOMS_df_sample['VAERS_ID'], VAERSDATA_2023_with_SYMPTOMS_df_sample['SYMPTOM_TEXT'])).unique()
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Resp-ID'] = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['ID_txt_zip'].map(lambda x: x[0])
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Symptom_txt'] = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['ID_txt_zip'].map(lambda x: x[1])
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned.drop('ID_txt_zip', axis = 1)

for LLT in VAERSDATA_2023_with_SYMPTOMS_value_counts_100_up:
    VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned[LLT] = 0
    
for ID in VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Resp-ID'].unique():
    for LLT in VAERSDATA_2023_with_SYMPTOMS_df_sample[VAERSDATA_2023_with_SYMPTOMS_df_sample['VAERS_ID'] == ID]['SYMPTOM'].unique():
        if LLT in VAERSDATA_2023_with_SYMPTOMS_value_counts_100_up:
            VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned.loc[VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Resp-ID'] == ID, LLT] = 1


C:\Users\sanka\AppData\Local\Temp\ipykernel_25808\3658117148.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned[LLT] = 0
C:\Users\sanka\AppData\Local\Temp\ipykernel_25808\3658117148.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned[LLT] = 0
C:\Users\sanka\AppData\Local\Temp\ipykernel_25808\3658117148.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has p

In [21]:
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned.shape

(19992, 114)

In [22]:
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned[VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Symptom_txt'] == 'BLANK']

,Resp-ID,Symptom_txt,Product storage error,COVID-19,No adverse event,SARS-CoV-2 test positive,Expired product administered,Fatigue,Pyrexia,Headache,...,Electrocardiogram abnormal,Acute kidney injury,Full blood count,Confusional state,Impaired work ability,Sepsis,Loss of consciousness,Swelling,Sneezing,Incomplete course of vaccination
1004,2549054,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5670,2556235,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7672,2559854,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9365,2562417,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9885,2563667,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10222,2564515,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12047,2567673,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14720,2572234,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15840,2574274,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15870,2574304,BLANK,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
tokenized = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Symptom_txt'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 512, truncation = True)))

In [25]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])  

In [26]:
np.array(padded).shape

(19992, 512)

In [27]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(19992, 512)

In [28]:
padded = padded
attention_mask = attention_mask

In [29]:
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned.to_csv('VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned.csv', index = False)

In [31]:
#last_hidden_states_list = []

for i in range(int(np.ceil(padded.shape[0]/100))):
    input_ids_iter = torch.tensor(padded[(100*i):(100*(i + 1))])  
    attention_mask_iter = torch.tensor(attention_mask[(100*i):(100*(i + 1))])
    
    with torch.no_grad():
        last_hidden_states = model(input_ids_iter, attention_mask=attention_mask_iter)
        
    if i == 0:
        df_out = pd.DataFrame(np.average(last_hidden_states.last_hidden_state.numpy(), axis = 1))
        df_out['Resp-ID'] = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Resp-ID'][0:100].tolist()
        df_out.to_csv('VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_BERT.csv', index = False)
        
    else:
        df_out = pd.DataFrame(np.average(last_hidden_states.last_hidden_state.numpy(), axis = 1))
        df_out['Resp-ID'] = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned['Resp-ID'][(100*i):(100*(i + 1))].tolist()
        df_out.to_csv('VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_BERT.csv', mode = 'a', index = False, header = False)
        
    #last_hidden_states_list.append(last_hidden_states.last_hidden_state)
    
    print('Iteration ' + str((i + 1)) + ' is completed at ' + str(datetime.datetime.now()))

Iteration 1 is completed at 2023-03-01 20:01:35.660460
Iteration 2 is completed at 2023-03-01 20:02:27.411696
Iteration 3 is completed at 2023-03-01 20:03:23.694959
Iteration 4 is completed at 2023-03-01 20:04:21.020706
Iteration 5 is completed at 2023-03-01 20:05:17.990241
Iteration 6 is completed at 2023-03-01 20:06:12.842853
Iteration 7 is completed at 2023-03-01 20:07:11.302763
Iteration 8 is completed at 2023-03-01 20:08:07.322710
Iteration 9 is completed at 2023-03-01 20:09:02.925496
Iteration 10 is completed at 2023-03-01 20:09:58.655667
Iteration 11 is completed at 2023-03-01 20:10:54.923555
Iteration 12 is completed at 2023-03-01 20:11:49.532466
Iteration 13 is completed at 2023-03-01 20:12:45.900845
Iteration 14 is completed at 2023-03-01 20:13:39.171469
Iteration 15 is completed at 2023-03-01 20:14:31.530637
Iteration 16 is completed at 2023-03-01 20:15:26.446659
Iteration 17 is completed at 2023-03-01 20:16:21.309691
Iteration 18 is completed at 2023-03-01 20:17:16.798663
I

Iteration 147 is completed at 2023-03-01 22:05:43.287023
Iteration 148 is completed at 2023-03-01 22:06:33.048518
Iteration 149 is completed at 2023-03-01 22:07:23.886322
Iteration 150 is completed at 2023-03-01 22:08:10.921053
Iteration 151 is completed at 2023-03-01 22:08:57.757338
Iteration 152 is completed at 2023-03-01 22:09:48.053673
Iteration 153 is completed at 2023-03-01 22:10:39.174081
Iteration 154 is completed at 2023-03-01 22:11:30.460795
Iteration 155 is completed at 2023-03-01 22:12:22.027105
Iteration 156 is completed at 2023-03-01 22:13:12.385107
Iteration 157 is completed at 2023-03-01 22:13:58.710152
Iteration 158 is completed at 2023-03-01 22:14:49.554942
Iteration 159 is completed at 2023-03-01 22:15:39.440859
Iteration 160 is completed at 2023-03-01 22:16:28.537053
Iteration 161 is completed at 2023-03-01 22:17:13.287856
Iteration 162 is completed at 2023-03-01 22:18:03.495163
Iteration 163 is completed at 2023-03-01 22:18:52.895640
Iteration 164 is completed at 2

In [32]:
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_BERT = pd.read_csv('VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_BERT.csv')

In [33]:
VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_all = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned.set_index('Resp-ID').join(VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_BERT.set_index('Resp-ID'))

In [35]:
features = VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_all[[str(i) for i in range(768)]]

In [38]:
model_results_dataframe_data_list = []

for LLT in VAERSDATA_2023_with_SYMPTOMS_value_counts_100_up:
    train_features, test_features, train_labels, test_labels = train_test_split(features, VAERSDATA_2023_with_SYMPTOMS_df_sample_cleaned_all[LLT])
    lr_clf = LogisticRegression(max_iter=3000)
    lr_clf.fit(train_features, train_labels)
    
    fpr, tpr, thresholds = roc_curve(test_labels, lr_clf.predict(test_features), pos_label=1)

    
    print('For ' + LLT + ', The efficacy of the model is (out of 1) ' + str(lr_clf.score(test_features, test_labels)))
    
    print('For ' + LLT + ', Accuracy is ' + str(accuracy_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', Recall is ' + str(recall_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', Precision is ' + str(precision_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', F1-Score is ' + str(f1_score(test_labels, lr_clf.predict(test_features))))
    print('For ' + LLT + ', AUC is ' + str(auc(fpr, tpr)))#
    
    model_results_dataframe_data_list.append([LLT, round(lr_clf.score(test_features, test_labels), 4), round(accuracy_score(test_labels, lr_clf.predict(test_features)), 4), round(recall_score(test_labels, lr_clf.predict(test_features)), 4),
                               round(precision_score(test_labels, lr_clf.predict(test_features)), 4), round(f1_score(test_labels, lr_clf.predict(test_features)), 4),
                               round(auc(fpr, tpr), 4)])

For Product storage error, The efficacy of the model is (out of 1) 0.9921968787515006
For Product storage error, Accuracy is 0.9921968787515006
For Product storage error, Recall is 0.9735812133072407
For Product storage error, Precision is 0.9880834160873883
For Product storage error, F1-Score is 0.980778708723509
For Product storage error, AUC is 0.9852815523276647
For COVID-19, The efficacy of the model is (out of 1) 0.9603841536614646
For COVID-19, Accuracy is 0.9603841536614646
For COVID-19, Recall is 0.8861439312567132
For COVID-19, Precision is 0.8996728462377317
For COVID-19, F1-Score is 0.8928571428571428
For COVID-19, AUC is 0.9317614173126447
For No adverse event, The efficacy of the model is (out of 1) 0.9723889555822329
For No adverse event, Accuracy is 0.9723889555822329
For No adverse event, Recall is 0.9136522753792299
For No adverse event, Precision is 0.9244391971664699
For No adverse event, F1-Score is 0.9190140845070423
For No adverse event, AUC is 0.9490985356611195

For Vaccine breakthrough infection, The efficacy of the model is (out of 1) 0.9985994397759104
For Vaccine breakthrough infection, Accuracy is 0.9985994397759104
For Vaccine breakthrough infection, Recall is 0.94
For Vaccine breakthrough infection, Precision is 0.9894736842105263
For Vaccine breakthrough infection, F1-Score is 0.964102564102564
For Vaccine breakthrough infection, AUC is 0.969897917517354
For Wrong product administered, The efficacy of the model is (out of 1) 0.985594237695078
For Wrong product administered, Accuracy is 0.985594237695078
For Wrong product administered, Recall is 0.41818181818181815
For Wrong product administered, Precision is 0.8518518518518519
For Wrong product administered, F1-Score is 0.5609756097560975
For Wrong product administered, AUC is 0.7082725784853445
For Arthralgia, The efficacy of the model is (out of 1) 0.9809923969587835
For Arthralgia, Accuracy is 0.9809923969587835
For Arthralgia, Recall is 0.21495327102803738
For Arthralgia, Precision

For Mobility decreased, The efficacy of the model is (out of 1) 0.9895958383353342
For Mobility decreased, Accuracy is 0.9895958383353342
For Mobility decreased, Recall is 0.22033898305084745
For Mobility decreased, Precision is 0.6842105263157895
For Mobility decreased, F1-Score is 0.3333333333333333
For Mobility decreased, AUC is 0.6095620811184588
For Product preparation issue, The efficacy of the model is (out of 1) 0.9961984793917567
For Product preparation issue, Accuracy is 0.9961984793917567
For Product preparation issue, Recall is 0.7083333333333334
For Product preparation issue, Precision is 0.8717948717948718
For Product preparation issue, F1-Score is 0.7816091954022988
For Product preparation issue, AUC is 0.8536616161616163
For Cerebrovascular accident, The efficacy of the model is (out of 1) 0.990796318527411
For Cerebrovascular accident, Accuracy is 0.990796318527411
For Cerebrovascular accident, Recall is 0.15217391304347827
For Cerebrovascular accident, Precision is 0.

For Gait disturbance, The efficacy of the model is (out of 1) 0.9905962384953981
For Gait disturbance, Accuracy is 0.9905962384953981
For Gait disturbance, Recall is 0.06818181818181818
For Gait disturbance, Precision is 0.3333333333333333
For Gait disturbance, F1-Score is 0.11320754716981131
For Gait disturbance, AUC is 0.533485337835358
For Nasopharyngitis, The efficacy of the model is (out of 1) 0.9921968787515006
For Nasopharyngitis, Accuracy is 0.9921968787515006
For Nasopharyngitis, Recall is 0.0
For Nasopharyngitis, Precision is 0.0
For Nasopharyngitis, F1-Score is 0.0
For Nasopharyngitis, AUC is 0.4997984277363435
For Hyperhidrosis, The efficacy of the model is (out of 1) 0.9935974389755903
For Hyperhidrosis, Accuracy is 0.9935974389755903
For Hyperhidrosis, Recall is 0.16666666666666666
For Hyperhidrosis, Precision is 0.75
For Hyperhidrosis, F1-Score is 0.27272727272727276
For Hyperhidrosis, AUC is 0.5831318016928658
For Influenza, The efficacy of the model is (out of 1) 0.995

C:\Users\sanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

For Echocardiogram abnormal, The efficacy of the model is (out of 1) 0.9943977591036415
For Echocardiogram abnormal, Accuracy is 0.9943977591036415
For Echocardiogram abnormal, Recall is 0.0
For Echocardiogram abnormal, Precision is 0.0
For Echocardiogram abnormal, F1-Score is 0.0
For Echocardiogram abnormal, AUC is 0.499899416616375
For Electrocardiogram abnormal, The efficacy of the model is (out of 1) 0.9953981592637055
For Electrocardiogram abnormal, Accuracy is 0.9953981592637055
For Electrocardiogram abnormal, Recall is 0.08333333333333333
For Electrocardiogram abnormal, Precision is 0.6666666666666666
For Electrocardiogram abnormal, F1-Score is 0.14814814814814814
For Electrocardiogram abnormal, AUC is 0.5415661439485323
For Acute kidney injury, The efficacy of the model is (out of 1) 0.9957983193277311
For Acute kidney injury, Accuracy is 0.9957983193277311
For Acute kidney injury, Recall is 0.1111111111111111
For Acute kidney injury, Precision is 0.2857142857142857
For Acute k

C:\Users\sanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


For Confusional state, The efficacy of the model is (out of 1) 0.9957983193277311
For Confusional state, Accuracy is 0.9957983193277311
For Confusional state, Recall is 0.08695652173913043
For Confusional state, Precision is 1.0
For Confusional state, F1-Score is 0.16
For Confusional state, AUC is 0.5434782608695652
For Impaired work ability, The efficacy of the model is (out of 1) 0.9957983193277311
For Impaired work ability, Accuracy is 0.9957983193277311
For Impaired work ability, Recall is 0.05
For Impaired work ability, Precision is 0.3333333333333333
For Impaired work ability, F1-Score is 0.08695652173913045
For Impaired work ability, AUC is 0.5247991161108879
For Sepsis, The efficacy of the model is (out of 1) 0.9939975990396158
For Sepsis, Accuracy is 0.9939975990396158
For Sepsis, Recall is 0.0
For Sepsis, Precision is 0.0
For Sepsis, F1-Score is 0.0
For Sepsis, AUC is 0.5


C:\Users\sanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sanka\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


For Loss of consciousness, The efficacy of the model is (out of 1) 0.993797519007603
For Loss of consciousness, Accuracy is 0.993797519007603
For Loss of consciousness, Recall is 0.03571428571428571
For Loss of consciousness, Precision is 0.2
For Loss of consciousness, F1-Score is 0.06060606060606061
For Loss of consciousness, AUC is 0.5174547283702213
For Swelling, The efficacy of the model is (out of 1) 0.9949979991996799
For Swelling, Accuracy is 0.9949979991996799
For Swelling, Recall is 0.038461538461538464
For Swelling, Precision is 1.0
For Swelling, F1-Score is 0.07407407407407407
For Swelling, AUC is 0.5192307692307693
For Sneezing, The efficacy of the model is (out of 1) 0.9957983193277311
For Sneezing, Accuracy is 0.9957983193277311
For Sneezing, Recall is 0.16
For Sneezing, Precision is 1.0
For Sneezing, F1-Score is 0.2758620689655173
For Sneezing, AUC is 0.58
For Incomplete course of vaccination, The efficacy of the model is (out of 1) 0.9991996798719488
For Incomplete cour

In [40]:
# Step to write out the data
pd.DataFrame(model_results_dataframe_data_list, columns = ['LLT', 'MODEL_SCORE', 'ACCURACY', 'RECALL', 'PRECISION', 'F1-SCORE', 'AUC']).to_csv('VAERSDATA_2023_modeling_metrics.csv')

In [41]:
model_results_dataframe_data_list

[['Product storage error', 0.9922, 0.9922, 0.9736, 0.9881, 0.9808, 0.9853],
 ['COVID-19', 0.9604, 0.9604, 0.8861, 0.8997, 0.8929, 0.9318],
 ['No adverse event', 0.9724, 0.9724, 0.9137, 0.9244, 0.919, 0.9491],
 ['SARS-CoV-2 test positive', 0.9674, 0.9674, 0.8359, 0.8821, 0.8584, 0.9104],
 ['Expired product administered',
  0.992,
  0.992,
  0.9339,
  0.9739,
  0.9535,
  0.9658],
 ['Fatigue', 0.9626, 0.9626, 0.5128, 0.8205, 0.6312, 0.7527],
 ['Pyrexia', 0.965, 0.965, 0.5209, 0.7366, 0.6102, 0.7553],
 ['Headache', 0.9636, 0.9636, 0.4926, 0.7528, 0.5956, 0.7417],
 ['Cough', 0.9764, 0.9764, 0.6706, 0.8284, 0.7412, 0.8316],
 ['Incorrect product formulation administered',
  0.9876,
  0.9876,
  0.815,
  0.9024,
  0.8565,
  0.9054],
 ['Pain', 0.9558, 0.9558, 0.2571, 0.6176, 0.3631, 0.6245],
 ['Drug ineffective', 0.9948, 0.9948, 0.9526, 0.9263, 0.9393, 0.9746],
 ['SARS-CoV-2 test', 0.9854, 0.9854, 0.7291, 0.8916, 0.8022, 0.8627],
 ['Dyspnoea', 0.9742, 0.9742, 0.4821, 0.7705, 0.5931, 0.7381],
 ['

In [42]:
pd.DataFrame(model_results_dataframe_data_list, columns = ['LLT', 'MODEL_SCORE', 'ACCURACY', 'RECALL', 'PRECISION', 'F1-SCORE', 'AUC'])

,LLT,MODEL_SCORE,ACCURACY,RECALL,PRECISION,F1-SCORE,AUC
0,Product storage error,0.9922,0.9922,0.9736,0.9881,0.9808,0.9853
1,COVID-19,0.9604,0.9604,0.8861,0.8997,0.8929,0.9318
2,No adverse event,0.9724,0.9724,0.9137,0.9244,0.9190,0.9491
3,SARS-CoV-2 test positive,0.9674,0.9674,0.8359,0.8821,0.8584,0.9104
4,Expired product administered,0.9920,0.9920,0.9339,0.9739,0.9535,0.9658
...,...,...,...,...,...,...,...
107,Sepsis,0.9940,0.9940,0.0000,0.0000,0.0000,0.5000
108,Loss of consciousness,0.9938,0.9938,0.0357,0.2000,0.0606,0.5175
109,Swelling,0.9950,0.9950,0.0385,1.0000,0.0741,0.5192
110,Sneezing,0.9958,0.9958,0.1600,1.0000,0.2759,0.5800
